In [0]:
orders = spark.read.option('inferSchema','true').csv('/FileStore/tables/part_00000')

In [0]:
orders = orders.toDF('order_customer_id','order_date','order_id','order_status')

In [0]:
orders.show()

+-----------------+-------------------+--------+---------------+
|order_customer_id|         order_date|order_id|   order_status|
+-----------------+-------------------+--------+---------------+
|                1|2013-07-25 00:00:00|   11599|         CLOSED|
|                2|2013-07-25 00:00:00|     256|PENDING_PAYMENT|
|                3|2013-07-25 00:00:00|   12111|       COMPLETE|
|                4|2013-07-25 00:00:00|    8827|         CLOSED|
|                5|2013-07-25 00:00:00|   11318|       COMPLETE|
|                6|2013-07-25 00:00:00|    7130|       COMPLETE|
|                7|2013-07-25 00:00:00|    4530|       COMPLETE|
|                8|2013-07-25 00:00:00|    2911|     PROCESSING|
|                9|2013-07-25 00:00:00|    5657|PENDING_PAYMENT|
|               10|2013-07-25 00:00:00|    5648|PENDING_PAYMENT|
|               11|2013-07-25 00:00:00|     918| PAYMENT_REVIEW|
|               12|2013-07-25 00:00:00|    1837|         CLOSED|
|               13|2013-0

Here are the common aggregate functions that are available as part of `pyspark.sql.functions`
* `count`
* `sum`
* `min`
* `max`
* `avg`

Spark also supports some statistical functions such as stddev.

In [0]:
from pyspark.sql.functions import count

In [0]:
help(count)

Help on function count in module pyspark.sql.functions:

count(col)
    Aggregate function: returns the number of items in a group.
    
    .. versionadded:: 1.3



In [0]:
orders.select(count("*")).show()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [0]:
orders.groupBy('order_status').agg(count("*")).show()

+---------------+--------+
|   order_status|count(1)|
+---------------+--------+
|PENDING_PAYMENT|   15030|
|       COMPLETE|   22899|
|        ON_HOLD|    3798|
| PAYMENT_REVIEW|     729|
|     PROCESSING|    8275|
|         CLOSED|    7556|
|SUSPECTED_FRAUD|    1558|
|        PENDING|    7610|
|       CANCELED|    1428|
+---------------+--------+



In [0]:
order_items = spark.read.option('inferSchema','true').csv('/FileStore/tables/part_00000-1')

In [0]:
order_items = order_items.toDF('id','order_id','product_id','quantity','product_price','subtotal')

In [0]:
order_items.show()

+---+--------+----------+--------+-------------+--------+
| id|order_id|product_id|quantity|product_price|subtotal|
+---+--------+----------+--------+-------------+--------+
|  1|       1|       957|       1|       299.98|  299.98|
|  2|       2|      1073|       1|       199.99|  199.99|
|  3|       2|       502|       5|        250.0|    50.0|
|  4|       2|       403|       1|       129.99|  129.99|
|  5|       4|       897|       2|        49.98|   24.99|
|  6|       4|       365|       5|       299.95|   59.99|
|  7|       4|       502|       3|        150.0|    50.0|
|  8|       4|      1014|       4|       199.92|   49.98|
|  9|       5|       957|       1|       299.98|  299.98|
| 10|       5|       365|       5|       299.95|   59.99|
| 11|       5|      1014|       2|        99.96|   49.98|
| 12|       5|       957|       1|       299.98|  299.98|
| 13|       5|       403|       1|       129.99|  129.99|
| 14|       7|      1073|       1|       199.99|  199.99|
| 15|       7|

* Get revenue using `subtotal` for a given `order_id`(eg:2)

In [0]:
from pyspark.sql.functions import sum

In [0]:
order_items.filter('order_id = 2').select(sum('subtotal').alias('order_revenue')).show()

+-------------+
|order_revenue|
+-------------+
|       379.98|
+-------------+



* Get number of items, total quantity as well as revenue for a given order item order id (eg:2)
    * Number of items can be computed using `count` on `quantity`.
    * Total quantity can be computed using `sum` on `quantity`.
    * Total Revenue can be computed using `sum` on `subtotal`

In [0]:
order_items.\
  filter('order_id = 2').\
  select(
        count('quantity').alias('order_item_count'),
        sum('quantity').alias('order_quantity'),
        sum('subtotal').alias('order_revenue')
        ).show()

+----------------+--------------+-------------+
|order_item_count|order_quantity|order_revenue|
+----------------+--------------+-------------+
|               3|             7|       379.98|
+----------------+--------------+-------------+



In [0]:
order_items.count()

Out[102]: 172198

In [0]:
order_items.select(count('*')) # This will return dataFrame

Out[103]: DataFrame[count(1): bigint]

In [0]:
# count is a wide transformation.
# Execution will be triggered when we perform actions such as show
order_items.select(count('*')).show()


+--------+
|count(1)|
+--------+
|  172198|
+--------+



In [0]:
help(order_items.groupBy)

Help on method groupBy in module pyspark.sql.dataframe:

groupBy(*cols) method of pyspark.sql.dataframe.DataFrame instance
    Groups the :class:`DataFrame` using the specified columns,
    so we can run aggregation on them. See :class:`GroupedData`
    for all the available aggregate functions.
    
    :func:`groupby` is an alias for :func:`groupBy`.
    
    .. versionadded:: 1.3.0
    
    Parameters
    ----------
    cols : list, str or :class:`Column`
        columns to group by.
        Each element should be a column name (string) or an expression (:class:`Column`).
    
    Examples
    --------
    >>> df.groupBy().avg().collect()
    [Row(avg(age)=3.5)]
    >>> sorted(df.groupBy('name').agg({'age': 'mean'}).collect())
    [Row(name='Alice', avg(age)=2.0), Row(name='Bob', avg(age)=5.0)]
    >>> sorted(df.groupBy(df.name).avg().collect())
    [Row(name='Alice', avg(age)=2.0), Row(name='Bob', avg(age)=5.0)]
    >>> sorted(df.groupBy(['name', df.age]).count().collect())
    [Ro

In [0]:
order_items.dtypes

Out[106]: [('id', 'int'),
 ('order_id', 'int'),
 ('product_id', 'int'),
 ('quantity', 'int'),
 ('product_price', 'double'),
 ('subtotal', 'double')]

In [0]:
order_items.groupBy().min().show()

+-------+-------------+---------------+-------------+------------------+-------------+
|min(id)|min(order_id)|min(product_id)|min(quantity)|min(product_price)|min(subtotal)|
+-------+-------------+---------------+-------------+------------------+-------------+
|      1|            1|             19|            1|              9.99|         9.99|
+-------+-------------+---------------+-------------+------------------+-------------+



In [0]:
orders.dtypes

Out[108]: [('order_customer_id', 'int'),
 ('order_date', 'timestamp'),
 ('order_id', 'int'),
 ('order_status', 'string')]

In [0]:
orders.groupBy().min().show()

+----------------------+-------------+
|min(order_customer_id)|min(order_id)|
+----------------------+-------------+
|                     1|            1|
+----------------------+-------------+



In [0]:
order_items_grouped = order_items.groupBy('order_id')

In [0]:
type(order_items_grouped)

Out[114]: pyspark.sql.group.GroupedData

In [0]:
order_items_grouped.\
      count().\
      withColumnRenamed('count','order_count').\
show()

+--------+-----------+
|order_id|order_count|
+--------+-----------+
|     148|          3|
|     463|          4|
|     471|          2|
|     496|          5|
|    1088|          2|
|    1580|          1|
|    1591|          3|
|    1645|          5|
|    2366|          1|
|    2659|          5|
|    2866|          4|
|    3175|          2|
|    3749|          1|
|    3794|          1|
|    3918|          4|
|    3997|          2|
|    4101|          1|
|    4519|          1|
|    4818|          1|
|    4900|          2|
+--------+-----------+
only showing top 20 rows



In [0]:
# get the sum of all numeric fields
order_items_grouped.\
  sum().show()

+--------+-------+-------------+---------------+-------------+------------------+------------------+
|order_id|sum(id)|sum(order_id)|sum(product_id)|sum(quantity)|sum(product_price)|     sum(subtotal)|
+--------+-------+-------------+---------------+-------------+------------------+------------------+
|     148|   1047|          444|           1407|            8|            479.99|            229.99|
|     463|   4522|         1852|           1685|           13| 829.9200000000001|249.97000000000003|
|     471|   2307|          942|           1030|            2|169.98000000000002|169.98000000000002|
|     496|   6125|         2480|           3105|            7|441.95000000000005|341.95000000000005|
|    1088|   5407|         2176|            768|            3|249.97000000000003|189.98000000000002|
|    1580|   3944|         1580|             44|            5|            299.95|             59.99|
|    1591|  11907|         4773|           2268|           10|            439.86|          

In [0]:
# get sum of all numeric fields
#Ignored order_date and order_status as they are not numeric fields

orders.groupBy('order_date').sum().show()

+-------------------+----------------------+-------------+
|         order_date|sum(order_customer_id)|sum(order_id)|
+-------------------+----------------------+-------------+
|2013-10-31 00:00:00|               4182458|      1226858|
|2013-09-22 00:00:00|               3610879|      1193465|
|2014-01-20 00:00:00|               7124675|      1222364|
|2013-08-25 00:00:00|               2131835|      1332197|
|2014-05-25 00:00:00|               6411998|       748321|
|2013-09-15 00:00:00|               2978157|       891561|
|2013-10-11 00:00:00|               4390266|      1401226|
|2014-07-08 00:00:00|              14535958|      1572287|
|2013-08-05 00:00:00|               1458309|       890288|
|2013-09-21 00:00:00|               2446415|      1034456|
|2014-03-14 00:00:00|               8252981|      1220792|
|2013-08-20 00:00:00|               1971999|      1154645|
|2014-02-01 00:00:00|               9864587|      1713675|
|2014-03-27 00:00:00|               6565845|      103147

In [0]:
order_items_grouped = order_items.\
  select('order_id','quantity','subtotal').\
  groupBy('order_id')

In [0]:
# gets sum on order_id as well
# it is not relevvant and better to discard aggregation on key fields such as order_id 

order_items_grouped.sum().show()

+--------+-------------+-------------+------------------+
|order_id|sum(order_id)|sum(quantity)|     sum(subtotal)|
+--------+-------------+-------------+------------------+
|     148|          444|            8|            229.99|
|     463|         1852|           13|249.97000000000003|
|     471|          942|            2|169.98000000000002|
|     496|         2480|            7|341.95000000000005|
|    1088|         2176|            3|189.98000000000002|
|    1580|         1580|            5|             59.99|
|    1591|         4773|           10|            129.96|
|    1645|         8225|           14| 989.9200000000001|
|    2366|         2366|            3|             99.99|
|    2659|        13295|            8| 624.9300000000001|
|    2866|        11464|            5|            519.96|
|    3175|         6350|            4| 89.99000000000001|
|    3749|         3749|            3|             47.99|
|    3794|         3794|            5|             59.99|
|    3918|    

In [0]:
# Consider only quantity and subtotal
order_items_grouped.sum('quantity','subtotal').show()

+--------+-------------+------------------+
|order_id|sum(quantity)|     sum(subtotal)|
+--------+-------------+------------------+
|     148|            8|            229.99|
|     463|           13|249.97000000000003|
|     471|            2|169.98000000000002|
|     496|            7|341.95000000000005|
|    1088|            3|189.98000000000002|
|    1580|            5|             59.99|
|    1591|           10|            129.96|
|    1645|           14| 989.9200000000001|
|    2366|            3|             99.99|
|    2659|            8| 624.9300000000001|
|    2866|            5|            519.96|
|    3175|            4| 89.99000000000001|
|    3749|            3|             47.99|
|    3794|            5|             59.99|
|    3918|            9|489.96000000000004|
|    3997|            4|            459.97|
|    4101|            1|            129.99|
|    4519|            2|             39.99|
|    4818|            1|            399.98|
|    4900|            2|        

In [0]:
order_items_grouped.sum('quantity','subtotal').\
    toDF('order_id','quantity','revenue').show()

+--------+--------+------------------+
|order_id|quantity|           revenue|
+--------+--------+------------------+
|     148|       8|            229.99|
|     463|      13|249.97000000000003|
|     471|       2|169.98000000000002|
|     496|       7|341.95000000000005|
|    1088|       3|189.98000000000002|
|    1580|       5|             59.99|
|    1591|      10|            129.96|
|    1645|      14| 989.9200000000001|
|    2366|       3|             99.99|
|    2659|       8| 624.9300000000001|
|    2866|       5|            519.96|
|    3175|       4| 89.99000000000001|
|    3749|       3|             47.99|
|    3794|       5|             59.99|
|    3918|       9|489.96000000000004|
|    3997|       4|            459.97|
|    4101|       1|            129.99|
|    4519|       2|             39.99|
|    4818|       1|            399.98|
|    4900|       2|            179.97|
+--------+--------+------------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import round

In [0]:
order_items_grouped.\
  sum('quantity','subtotal').\
  toDF('order_id','quantity','revenue').\
  withColumn('revenue',round('revenue',2)).show()

+--------+--------+-------+
|order_id|quantity|revenue|
+--------+--------+-------+
|     148|       8| 229.99|
|     463|      13| 249.97|
|     471|       2| 169.98|
|     496|       7| 341.95|
|    1088|       3| 189.98|
|    1580|       5|  59.99|
|    1591|      10| 129.96|
|    1645|      14| 989.92|
|    2366|       3|  99.99|
|    2659|       8| 624.93|
|    2866|       5| 519.96|
|    3175|       4|  89.99|
|    3749|       3|  47.99|
|    3794|       5|  59.99|
|    3918|       9| 489.96|
|    3997|       4| 459.97|
|    4101|       1| 129.99|
|    4519|       2|  39.99|
|    4818|       1| 399.98|
|    4900|       2| 179.97|
+--------+--------+-------+
only showing top 20 rows



In [0]:
order_items.groupBy('order_id').sum('quantity','subtotal').show()

+--------+-------------+------------------+
|order_id|sum(quantity)|     sum(subtotal)|
+--------+-------------+------------------+
|     148|            8|            229.99|
|     463|           13|249.97000000000003|
|     471|            2|169.98000000000002|
|     496|            7|341.95000000000005|
|    1088|            3|189.98000000000002|
|    1580|            5|             59.99|
|    1591|           10|            129.96|
|    1645|           14| 989.9200000000001|
|    2366|            3|             99.99|
|    2659|            8| 624.9300000000001|
|    2866|            5|            519.96|
|    3175|            4| 89.99000000000001|
|    3749|            3|             47.99|
|    3794|            5|             59.99|
|    3918|            9|489.96000000000004|
|    3997|            4|            459.97|
|    4101|            1|            129.99|
|    4519|            2|             39.99|
|    4818|            1|            399.98|
|    4900|            2|        

In [0]:
order_items.groupBy('order_id').agg(sum('quantity'),sum('subtotal')).show()

+--------+-------------+------------------+
|order_id|sum(quantity)|     sum(subtotal)|
+--------+-------------+------------------+
|     148|            8|            229.99|
|     463|           13|249.97000000000003|
|     471|            2|169.98000000000002|
|     496|            7|341.95000000000005|
|    1088|            3|189.98000000000002|
|    1580|            5|             59.99|
|    1591|           10|            129.96|
|    1645|           14| 989.9200000000001|
|    2366|            3|             99.99|
|    2659|            8| 624.9300000000001|
|    2866|            5|            519.96|
|    3175|            4| 89.99000000000001|
|    3749|            3|             47.99|
|    3794|            5|             59.99|
|    3918|            9|489.96000000000004|
|    3997|            4|            459.97|
|    4101|            1|            129.99|
|    4519|            2|             39.99|
|    4818|            1|            399.98|
|    4900|            2|        

In [0]:
order_items.groupBy('order_id').\
  agg(sum('quantity').alias('order_quantity'),
      round(sum('subtotal'),2).alias('order_revenue')).show()

+--------+--------------+-------------+
|order_id|order_quantity|order_revenue|
+--------+--------------+-------------+
|     148|             8|       229.99|
|     463|            13|       249.97|
|     471|             2|       169.98|
|     496|             7|       341.95|
|    1088|             3|       189.98|
|    1580|             5|        59.99|
|    1591|            10|       129.96|
|    1645|            14|       989.92|
|    2366|             3|        99.99|
|    2659|             8|       624.93|
|    2866|             5|       519.96|
|    3175|             4|        89.99|
|    3749|             3|        47.99|
|    3794|             5|        59.99|
|    3918|             9|       489.96|
|    3997|             4|       459.97|
|    4101|             1|       129.99|
|    4519|             2|        39.99|
|    4818|             1|       399.98|
|    4900|             2|       179.97|
+--------+--------------+-------------+
only showing top 20 rows



In [0]:
order_items.groupBy('order_id').\
  agg({'quantity':'sum','subtotal':'sum'}).show()

+--------+-------------+------------------+
|order_id|sum(quantity)|     sum(subtotal)|
+--------+-------------+------------------+
|     148|            8|            229.99|
|     463|           13|249.97000000000003|
|     471|            2|169.98000000000002|
|     496|            7|341.95000000000005|
|    1088|            3|189.98000000000002|
|    1580|            5|             59.99|
|    1591|           10|            129.96|
|    1645|           14| 989.9200000000001|
|    2366|            3|             99.99|
|    2659|            8| 624.9300000000001|
|    2866|            5|            519.96|
|    3175|            4| 89.99000000000001|
|    3749|            3|             47.99|
|    3794|            5|             59.99|
|    3918|            9|489.96000000000004|
|    3997|            4|            459.97|
|    4101|            1|            129.99|
|    4519|            2|             39.99|
|    4818|            1|            399.98|
|    4900|            2|        

In [0]:
order_items.groupBy('order_id').\
  agg({'quantity':'sum','subtotal':'sum'}).\
  toDF('order_id','order_quantity','order_revenue').\
  withColumn('order_revenue',round(col('order_revenue'),2)).\
show()

+--------+--------------+-------------+
|order_id|order_quantity|order_revenue|
+--------+--------------+-------------+
|     148|             8|       229.99|
|     463|            13|       249.97|
|     471|             2|       169.98|
|     496|             7|       341.95|
|    1088|             3|       189.98|
|    1580|             5|        59.99|
|    1591|            10|       129.96|
|    1645|            14|       989.92|
|    2366|             3|        99.99|
|    2659|             8|       624.93|
|    2866|             5|       519.96|
|    3175|             4|        89.99|
|    3749|             3|        47.99|
|    3794|             5|        59.99|
|    3918|             9|       489.96|
|    3997|             4|       459.97|
|    4101|             1|       129.99|
|    4519|             2|        39.99|
|    4818|             1|       399.98|
|    4900|             2|       179.97|
+--------+--------------+-------------+
only showing top 20 rows



In [0]:
# dict will expect unique column names , so in this case it will only print 1
order_items.groupBy('order_id').\
  agg({'quantity':'sum','quantity':'min'}).show()

+--------+-------------+
|order_id|min(quantity)|
+--------+-------------+
|     148|            1|
|     463|            1|
|     471|            1|
|     496|            1|
|    1088|            1|
|    1580|            5|
|    1591|            1|
|    1645|            1|
|    2366|            3|
|    2659|            1|
|    2866|            1|
|    3175|            1|
|    3749|            3|
|    3794|            5|
|    3918|            1|
|    3997|            1|
|    4101|            1|
|    4519|            2|
|    4818|            1|
|    4900|            1|
+--------+-------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import min

In [0]:
order_items.groupBy('order_id').\
  agg(
      sum('quantity').alias('order_quantity'),
      min('quantity').alias('min_order_quantity'),
      round(sum('subtotal'),2).alias('order_revenue'),
      min('subtotal').alias('min_subtotal')
     ).show()

+--------+--------------+------------------+-------------+------------+
|order_id|order_quantity|min_order_quantity|order_revenue|min_subtotal|
+--------+--------------+------------------+-------------+------------+
|     148|             8|                 1|       229.99|        50.0|
|     463|            13|                 1|       249.97|       39.99|
|     471|             2|                 1|       169.98|       39.99|
|     496|             7|                 1|       341.95|       49.98|
|    1088|             3|                 1|       189.98|       59.99|
|    1580|             5|                 5|        59.99|       59.99|
|    1591|            10|                 1|       129.96|       39.99|
|    1645|            14|                 1|       989.92|       39.99|
|    2366|             3|                 3|        99.99|       99.99|
|    2659|             8|                 1|       624.93|        25.0|
|    2866|             5|                 1|       519.96|      